# Photon Statistics
> Photon Statistics and the unintuitive way photons behave when they arrive sparsely

- toc: true 
- badges: false
- comments: true
- categories: [jupyter]
- permalink: /photon-statistics
- image: images/photon-statistics.png

Because of the inherent probability quantum mechanics entails, photons arriving at some detector follow statistical properties. In this chapter I will discuss these *Photon Statistics* and the unintuitive way photons behave when they arrive sparsely. 

## Photon Noise

In order to calculate the noise, we need to know the power that is impinging on the detector due to atmospheric loading. However, do take into account that when looking at an astronomical object far away, the detector is measuring photons that this galaxy randomly emits. This means that in the context of radio astronomy our signal is also photon noise{% cite perley_2008 %}! Therefore when I discuss noise in this chapter I both mean actual noise as is used in our signal to noise calculations and photon noise that is received as signal.

Photon noise can be split up into two categories: thermal noise and shot noise. In this section I will take a look at both 

### Thermal Noise

In radio receivers, a big part of the noise is generated by thermal agitation of electrons. In most uses this can be classified as white noise: the power is constant across its frequency spectrum {% cite turner_2007 %}, but in extremely high frequencies or low temperatures this approximation doesn't hold. The power $P$ transmitted by the noise is given as Johnson-Nyquist Noise and is approximated for small bandwidths $\Delta\nu$ as{% cite endo_session_4 %}:

$$\begin{equation}
P_\nu=\int_{\nu_0}^{\nu_1}\frac{h\nu}{e^{h\nu/k_BT}-1}d\nu \approx \frac{h\nu}{e^{h\nu/k_BT}-1}\Delta\nu
\end{equation}$$

Plotting this normalized ($P_\nu/k_BT$) power spectrum supports my earlier statements that, in  most cases, thermal noise can be approximated as white

In [20]:
#hide
!pip install -q galspec numpy pandas plotly opensimplex scipy

import numpy as np
import pandas as pd
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

from thesis_plot import plot

# Page specific
import random as rng
from opensimplex import OpenSimplex
from scipy.stats import cauchy
from scipy.special import gamma

In [21]:
#hide
h = 6.626e-34
k_B = 1.381e-23

In [22]:
#hide_input
nu=np.logspace(1,13,100)[np.newaxis].T
T=np.array([1,10,50,100,300])

P = h*nu/(np.exp(h*nu/(k_B*T))-1)

df = pd.DataFrame(P/(T*k_B))
df.columns = T
df['nu']=nu*1e-9
df.set_index('nu',inplace=True)

plot(df,axis_labels=('$\\nu\:\mathrm{[GHz]}$','$\\frac{P_\\nu}{k_BT}$','$T\:\mathrm{[K]}$'),hover_labels=('nu','normalized psd','T'),plot_type='linlog')



Unfortunately, this white noise approximation doesn't hold for the DESHIMA range. Once more, it is also not the only form of noise we have to deal with.

### Shot noise

When the source is very dim, photons arrive one at a time. This means that we are dealing with shot noise{% cite perley_2008%}. Here the detection rate of a photon is characterized by Poisson statistics and the famous Poisson distribution {% cite dekking_2011%}:

$$\begin{equation}
f(x)=\frac{\lambda^xe^{-\lambda}}{x!}
\end{equation}$$

Hence we also call this noise Poisson noise.

In [4]:
#hide_input
x=np.linspace(0,20,100)[np.newaxis].T
l=np.array([1,2,5,10])

f = np.power(l,x)*np.exp(-l)/gamma(x+1)

df = pd.DataFrame(f)
df.columns = l
df['x']=x
df.set_index('x',inplace=True)

plot(df,axis_labels=('$x$','$f(x)$','$\lambda$'),hover_labels=('x','f(x)','lambda'),plot_type='linlin',e_notation=[False,False])

### Uncertainty

From Poisson statistics we know that the uncertainty $\sigma_N$ over a number of measurements is equal to the square root of these measurements, since it's an uncorrelated process {% cite dekking_2011%}{% cite perley_2008%}. Because our power arrives with single photons hitting the detector, the power follows the same uncertainty

$$\begin{equation}
\sigma_N=\sqrt{N}
\end{equation}$$
$$\begin{equation}
\sigma_P \propto \sqrt{P}
\end{equation}$$

However, if we look at Johnson-Nyquist noise and approximate for lower frequencies we get{% cite perley_2008%}{% cite endo_session_4%}:

$$\begin{equation}
P \approx \frac{h\nu}{e^{h\nu/k_BT}-1}\Delta\nu \approx k_BT\Delta\nu
\end{equation}$$$$\begin{equation}
\sigma_P \propto P
\end{equation}$$

Hence we have found two proportionality relations for the uncertainty, which is it?

#### Particle-Wave duality

The seemingly paradoxical nature of this uncertainty can be explained as a caveat of the particle-wave duality. In the wave domain the Johnson-Nyquist noise dominates, whereas in the particle domain we get mainly Poisson noise. The general case of the uncertainty in the number of photons arriving in a detection time of $\tau$ is{% cite endo_memo_2020%}{% cite paul_2004%}:

$$\begin{equation}
\sigma_\mathrm{ph}= \frac{1}{\sqrt{\tau}}\sqrt{n^2+n}
\end{equation}$$

with $n$, the photon number, being the number of photons per time per bandwidth. Here both extremes are more obvious: for $n\gg 1$ we get the thermal noise from the wave domain and for $n \ll 1$ the Poisson uncertainty falls out.

![Radiaton density between the particle and the wave limit](images/Radhakrishnan_Radiation_Density.png)
{% cite perley_2008 %}*An analogy for the detection of photons in the particle and wave limit.*

The photon number for a thermal blackbody radiator is given by the Bose-Einstein equation {% cite zmuidzinas_2003%}{% cite paul_2004%}

$$\begin{equation}
n_\mathrm{th}\left(\nu,T\right)=\frac{1}{e^{h\nu/k_BT}-1}
\end{equation}$$

Which means that in the region between $220\:{\mathrm{GHz}}$ and $440\:{\mathrm{GHz}}$ for a relatively hot object the photon occupation number is neither fully in the wave limit nor the particle limit. 

In [24]:
#hide_input
nu=np.linspace(1.2e11,5.4e11,100)[np.newaxis].T
T=np.array([2, 5,10,25, 50])

n = 1/(np.exp(h*nu/(k_B*T))-1)

df = pd.DataFrame(n)
df.columns = T
df['nu']=nu*1e-9
df.set_index('nu',inplace=True)

plot(df,axis_labels=('$\\nu\:\mathrm{[GHz]}$','$n_\mathrm{th}$','$T\:\mathrm{[K]}$'),hover_labels=('nu','n','T'),plot_type='linlog')

Finally, another mechanism for noise needs discussing. Mainly in the wave domain, where the photon number is higher, photons tend to come clumped together in a process known as photon bunching{% cite paul_2004%}.

## Photon Bunching

Quantum Electrodynamics has taught us that photons behave stochastically, so it is reasonable to assume that photon detection also occurs randomly. While this is (obviously) true to some degree, in the previous section I stated photons can arrive 'clumped' together at the detector in a phenomenon known as photon bunching. This means that detecting one photon will result in a higher chance of another photon being detected within a specified time, called the coherence time $t_{\mathrm{coh}}$ {%cite paul_2004%}. In this chapter I will first present an intuitive, *qualitative* explanation as to why photon bunching occurs. Then I will go deeper into the mathematics and quantum mechanics of photon bunching and the specific case of the DESHIMA system.

### An analogy

To understand photon bunching, first think of photons arriving at the detector like raindrops falling on a piece of paper. While it's raining with constant intensity, the chance of a raindrop falling on a specific area within, say, the next second (expressed by $P(\mathrm{drop})$) is constant. In the figure below I have set $P(\mathrm{drop}) = 0.3$:

In [6]:
#hide

T = 10000
T_sel = 100
t = np.arange(0,T,1)
rand = np.random.rand(t.size)

def noise_array(length, scale):
    array = np.empty(length)
    tmp = OpenSimplex(seed=rng.getrandbits(32))
    for i in range(length):
        array[i] = tmp.noise2d(x=i*scale, y=10)
    return array

In [7]:
#hide_input
def plot_single(p_rnd):
    global t
    global rand
    global T_sel
    
    fig = px.scatter(pd.DataFrame((1+(rand[:T_sel]>p_rnd[:T_sel])*-2)*0.5),
                     labels = {'index': '$t$','value': '$P(\mathrm{drop})$'},
                     template = 'plotly_white')
    fig.update_traces(marker={'size': 20, 'opacity': 0.75},
                      hovertemplate= '<b>drop</b><br>Time: %{x}<extra></extra>')
    
    fig.add_scatter(x=t[:T_sel], y=p_rnd[:T_sel],line_shape='spline', hovertemplate= '<b>P(drop)</b><br>Time: %{x}<br>Probability: %{y:.2f}<extra></extra>')
    
    fig.update_layout(yaxis = {'range': [0,1.1], 'fixedrange': True},
                     showlegend=False,
                     hoverlabel={'bgcolor': "white", 'font_size': 14})
    fig.show()

In [8]:
#hide_input
def random_drops(T):
    
    p_rnd = np.ones(T)*0.3  
    return p_rnd

random = random_drops(T)
plot_single(random)

The raindrops fall uncorrelated: every second a new chance of $P(\mathrm{drop})$ means a raindrop falling from the sky might land in our defined area. This is how unbunched, or random, photons behave. 

Now let's assume the chance of a raindrop occurring is not constant over time, but rather varies relatively slowly over time. This could be because of varying wind speeds or varying intensity in the rain, but whatever the case the rain drops are no longer uncorrelated. There are moments of high intensity, where the chance of a raindrop is high and similarly there are moments of low intensity. I have simulated this using open simplex noise {% cite opensimplex %} in the following graph.

In [9]:
#hide_input
def noise_drops(T):
    p_rnd = noise_array(T,0.1)*0.35 + 0.35   
    return p_rnd

noise = noise_drops(T)
plot_single(noise)

As is clear from the graph, the raindrops are clumped together. This is the same underlying principle as photon bunching in photons from astronomical sources. The random motion of the exciting atoms, for example Brownian motion, means that the intensity also fluctuates over time{% cite paul_2004 %}. It is therefore not the act of detecting a photon that increases the chance of another detection, but rather an underlying change in probability of detecting, easily explained by probabilistics.

This means that at the point where a raindrop falls or a photon gets emitted, it is still a simple stochastic process. Should our sampling interval be much smaller than the time over which the rain varies, we'd still see 'stochastic regions', where the probability function stays roughly constant, like in the first example. This is illustrated in the graph below with stretched noise:

In [10]:
#hide_input
def incoherent_noise_drops(T):
    p_rnd = noise_array(T,0.01)*.35 + 0.35   
    return p_rnd

incoherent = incoherent_noise_drops(T)
plot_single(incoherent)

The droplets appear less bunched, because the time scale in which the measurements take place is much smaller than the time scale in which the intensity changes. The very first example I spoke about, of raindrops falling at a constant intensity, is a situation in which this discrepancy occurs. Of course it won't rain forever and when the rain dies down the chance of a raindrop falling is *much* lower than while it's still raining. So while the droplets look purely random when it rains, they aren't over the course of the entire day. I will call this behavior *bunched (slow timescale)*, because the sampling time is much faster than the coherence time, which I shall later elaborate on further.

Let's also take a look at what happens when the sampling time is much slower than the underlying probability fluctuations. 

In [11]:
#hide_input
def fast_noise_drops(T):
    p_rnd = noise_array(T,1)*.35 + 0.35   
    return p_rnd

fast = fast_noise_drops(T)
plot_single(fast)

As expected, the detection rate looks much like the first detection rate with constant $P\left(\mathrm{drop}\right)$. Although the underlying probability changes, it changes too fast between detections for bunching to occur.

#### Intermezzo: Antibunching

While this report focuses on photon bunching, the opposite also occurs in, for example, single photon sources {% cite Krottenmüller_2013 %}. Now that I have discussed a framework with which to explain photon bunching, I thought it would be remiss to leave out an explanation for antibunching. 

As one would expect, in antibunching the opposite of bunching happens: detecting a raindrop would mean that, at least for the next period of time, the chance of a raindrop falling becomes smaller. A good allegory for this is a dripping faucet. A small flow of water exits the faucet and, due to surface tension, pools up in a small droplet at the end. As the droplet gets bigger, the chance of it falling increases, but once it drips down it takes all water with it and the process starts anew.

In [12]:
#hide_input
def anti_drops(T):
    global rand
    
    p_slope=0.05
    
    p_rnd = np.zeros(T)
    
    p_rnd[0]=0
    for i in range(0,T-1):
        #print(rand[i])
        if rand[i]<p_rnd[i]:
           p_rnd[i+1] = 0
        else:
            p_rnd[i+1] = p_rnd[i] + p_slope
            #print(p_rnd)
    return p_rnd

anti = anti_drops(T)
plot_single(anti)

This behavior corresponds with antibunching behavior in single photon sources{% cite Krottenmüller_2013%}. Atoms are excited through an energy pump process. When this energy increases sufficiently they emit a photon and lose energy, ready to get excited again{% cite paul_2004 %}.

Again, the detection of such single photons will display antibunching, but it is not the detection that triggers this *cool-down* period, it is the emission.

In [13]:
#hide_input
drops = pd.DataFrame({'bunched': (1+(rand[:T_sel]>noise[:T_sel])*-2)*1.05, 'bunched (long timescale)': (1+(rand[:T_sel]>incoherent[:T_sel])*-2)*0.95,'bunched (short timescale)': (1+(rand[:T_sel]>fast[:T_sel])*-2)*0.85, 'random': (1+(rand[:T_sel]>random[:T_sel])*-2)*0.75, 'anti-bunched': (1+(rand[:T_sel]>anti[:T_sel])*-2)*0.65})

fig = px.scatter(drops,
                 labels = {'index': '$t$','value': '$P(\mathrm{drop})$', 'variable': 'Photon-mode'},
                 hover_data={'value': False},
                 template = 'plotly_white')

fig.update_traces(marker={'size': 20, 'opacity': 0.75},
                  hovertemplate= 'Time: %{x}<extra></extra>')

# Hacky update all traces because can't print trace name I guess
for trace in fig.data:
    trace['hovertemplate']= '<b>'+trace['name']+'</b><br>Time: %{x}<extra></extra>'

fig.add_scatter(x=t[:T_sel],
                y=noise[:T_sel],
                line_shape='spline',
                line = {'color': fig.data[0].marker.color},
                legendgroup = fig.data[0].legendgroup,
                showlegend=False,
                hovertemplate= '<b>'+fig.data[0].name+'</b><br>Time: %{x}<br>Probability: %{y:.2f}<extra></extra>')

fig.add_scatter(x=t[:T_sel],
                y=incoherent[:T_sel],
                line_shape='spline',
                line = {'color': fig.data[1].marker.color},
                legendgroup = fig.data[1].legendgroup,
                showlegend=False,
                hovertemplate= '<b>'+fig.data[1].name+'</b><br>Time: %{x}<br>Probability: %{y:.2f}<extra></extra>')

fig.add_scatter(x=t[:T_sel],
                y=fast[:T_sel],
                line_shape='spline',
                line = {'color': fig.data[2].marker.color},
                legendgroup = fig.data[2].legendgroup,
                showlegend=False,
                hovertemplate= '<b>'+fig.data[2].name+'</b><br>Time: %{x}<br>Probability: %{y:.2f}<extra></extra>')

fig.add_scatter(x=t[:T_sel],
                y=random[:T_sel],line_shape='spline',
                line = {'color': fig.data[3].marker.color},
                legendgroup = fig.data[3].legendgroup,
                showlegend=False,
               hovertemplate= '<b>'+fig.data[3].name+'</b><br>Time: %{x}<br>Probability: %{y:.2f}<extra></extra>')

fig.add_scatter(x=t[:T_sel],
                y=anti[:T_sel],
                line = {'color': fig.data[4].marker.color},
                legendgroup = fig.data[4].legendgroup,
                showlegend=False,
               hovertemplate= '<b>'+fig.data[4].name+'</b><br>Time: %{x}<br>Probability: %{y:.2f}<extra></extra>')


fig.update_layout(yaxis = {'range': [0,1.1], 'fixedrange': True},
                  hoverlabel = {'bgcolor': "white", 'font_size': 14})
fig.show()

### Coherence

If the underlying probability function is unknown (and quantum mechanically it is), how would we be able to quantify whether our detections are bunched. A possible heuristic we can use comes from signal processing. We could take the autocorrelation of the signals and compare them. Since the autocorrelation is the cross-correlation between the signal and a time-shifted copy, we'd likely see the bunched signal display some sort of correlation for a delay that is around the timescale of the oscillations in the probability function. In the figure below I have correlated the detections described above with $t\in[0,1000]$ and plotted these values for different time delays $h$

In [14]:
#hide_input
max_lag = 50
result = np.empty((5,max_lag))
for i, mode in enumerate([noise,incoherent,fast,random,anti]):
    autocor = np.empty(max_lag)
    series = pd.Series((rand>mode)*1.0)
    for h in range(1,max_lag):
        autocor[h] = series.autocorr(lag=h)
    result[i]=autocor

df = pd.DataFrame(result.T)
df.columns=['bunched','bunched (long timescale)', 'bunched (short timescale)', 'random','anti-bunched']
df.rename_axis('h',inplace=True)

fig = px.scatter(df,
                 labels={'value': '$\mathrm{R_{XX}}(h)$', 'h': '$h$', 'variable': 'photon-mode'},
                 template='plotly_white',
                 trendline='rolling',trendline_options={'window': 3})

for trace in fig.data:
    trace['hovertemplate']= '<b>'+trace['name']+'</b><br>Lag: %{x}<br>Autocorrelation: %{y:.2f}<extra></extra>'

fig.update_layout(hoverlabel = {'bgcolor': "white", 'font_size': 14})
fig.show()

As is clear from the graph, the bunched photons show some autocorrelation, with the longer timescale showing longer (bigger time shifts) autocorrelation. This can be explained further by signal analysis. We are working in the discrete domain and smaller sampling steps relative to the variation in the underlying probability will mean more correlation.

What this shows is temporal coherence, a property of light that describes the predictability over a timescale{% cite perley_2008 %}.

### Bandwidth

Up until now we have only looked at raindrops as analogous detections, but when talking about photons we need to take frequency into account. Photons obey the Heisenberg uncertainty relation where the uncertainty in position $\Delta x$ and momentum $\Delta p_x$ must obey {% cite perley_2008 %}:

$$\begin{equation}
\Delta x\Delta p_x = \frac{\hbar}{2}
\end{equation}$$

Our photons are flying at the detector, so it's uncertainty in longitudinal ($z$, towards the detector) position can be equated as it's uncertainty in detection time and it's momentum can be expressed as $h/\lambda$:

$$\begin{equation}
\Delta z \Delta p_z= \frac{\Delta t}{c} \frac{h}{\Delta\lambda} = \frac{\Delta t}{c} \frac{hc}{\Delta\nu} = \frac{\hbar}{2}
\end{equation}$$

and finally we arrive at

$$\begin{equation}
\Delta\nu \propto \frac{1}{\Delta t}
\end{equation}$$

In other words: a shorter coherence time means more accurate knowledge on where the photons are, meaning their uncertainty in momentum and therefore bandwidth is bigger.

This relation can also be seen experimentally. In Morgan and Mandel's 1966 paper{% cite morgan_1966%}, the authors explore the autocorrelation between two different light sources: a Hg<sup>198</sup> source with a very narrow bandwidth (a) and a tungsten light bulb with a broad spectrum thermal source (b). By designing an apparatus that counts the number of times two photon arrive within a defined time period, they show the autocorrelation of the two light sources

![Morgan and Mandel (1996)](images/photon_bunching_morgan_1966.png)
{% cite morgan_1966%}*The number of times per second two photons arrived after a specified time delay for a a) Hg<sup>198</sup> light source and a b) tungsten incandescent light source.*

Because the tungsten light has a high bandwidth, it has a short coherence time and the reverse is true for the Hg<sup>198</sup> light. Therefore, if we take the short timescale bunched photons as an analogy for the tungsten light and the longer timescale bunched lights as analogous for the mercury light, we get something that looks very similar.

In [15]:
#hide_input
fig = px.scatter(df[['bunched', 'bunched (short timescale)']],
                 labels={'value': '$\mathrm{R_{XX}}(h)$', 'h': '$h$', 'variable': 'photon-mode'},
                 template='plotly_white',
                 trendline='rolling',trendline_options={'window': 3})

for trace in fig.data:
    trace['hovertemplate']= '<b>'+trace['name']+'</b><br>Lag: %{x}<br>Autocorrelation: %{y:.2f}<extra></extra>'

fig.update_layout(hoverlabel = {'bgcolor': "white", 'font_size': 14})
fig.show()

With a more thorough understanding of photon bunching, it is time to return to photon noise and discuss noise-equivalent power.

## Noise Equivalent power

The noise equivalent power ($\mathrm{NEP}$) is a way to express the sensitivity of a photon detectors. It is defined as *the input signal power that results in a signal-to-noise-ratio of 1 in a 1 Hz output bandwidth*. {% cite richards_1994 %}. For our purposes this means multiplying the photon noise with the photon energy{% cite leclercq_2007%}. From {% cite zmuidzinas_2003%} we have an equation for the electrical (detected) photon noise

$$\begin{equation}
\sigma_\mathrm{ph}^2=\frac{1}{\tau}\int_0^\infty\eta(\nu)n(\nu)\left[1+\eta(\nu)n(\nu)\right]d\nu
\end{equation}$$

multiplying this integral with the photon energy, we get the electrical Noise Equivalent Power added by photon noise for a specified integration time

$$\begin{equation}
\mathrm{NEP}_{\tau,\mathrm{ph}}^2=\frac{1}{\tau}\int_0^\infty\left(h\nu\right)^2\eta(\nu)n(\nu)\left[1+\eta(\nu)n(\nu)\right]d\nu
\end{equation}$$

But unfortunately this integral is not easily solvable. As we've seen in the previous section, the coherence time is inversely proportional to the bandwidth and by taking an integral we effectively have an infinitesimal bandwidth and therefore an infinite coherence time.

If we were to create a perfect box filter, and approximate the photon number as constant over this filter we can get around this. Let's first define this box function

$$\begin{equation} \Pi^{\Delta\nu}_{\nu_0}(\nu)=   \left\{
\begin{array}{ll}
      1 & |\nu-\nu_0|\le\frac{\Delta\nu}{2} \\
      0 & \mathrm{elsewhere} \\
\end{array} 
\right.
\end{equation}$$

And then set $\eta(\nu)=\eta_0\Pi^{\Delta\nu}_{\nu_0}(\nu)$. When we assume we have a constant photon occupation number over this range, the equation simplifies to:

$$\begin{equation}
\begin{split}
\mathrm{NEP}_{\tau,\mathrm{ph}}^2&=\frac{1}{\tau}\int_0^\infty\left(h\nu\right)^2\eta_0\Pi^{\Delta\nu}_{\nu_0}(\nu)n(\nu)\left[1+\eta_0\Pi^{\Delta\nu}_{\nu_0}(\nu)n(\nu)\right]d\nu \\
&= \frac{1}{\tau}\int_{\nu_0-\frac{\Delta\nu}{2}}^{\nu_0+\frac{\Delta\nu}{2}}\left(h\nu\right)^2\eta_0n\left[1+\eta_0n\right]d\nu
\end{split}
\end{equation}$$

Which, for $\Delta\nu\ll\nu$ approximates as:
$$\begin{equation}
\mathrm{NEP}_{\tau,\mathrm{ph}}^2=\frac{1}{\tau}\left(h\nu_0\right)^2\eta_0n\left[1+\eta_0n\right]\Delta\nu
\end{equation}$$

In astrophysics it is convention to take an integration time of $\tau=0.5\:\mathrm{[s]}$ which results in a Noise Equivalent Power of:

$$\begin{equation}
\mathrm{NEP}_{\tau=0.5\mathrm{s},\mathrm{ph}}=h\nu_0\sqrt{2\eta_0n\left(1+\eta_0n\right)\Delta\nu}
\end{equation}$$

### DESHIMA

In actual measurements the photon occupation number isn't known, but we are able to deduce it from the power spectral density, as per Bose-Einstein statistics{% cite paul_2004 %}:

$$\begin{equation}
n_\mathrm{ph}=\frac{\mathrm{PSD}}{h\nu}
\end{equation}$$

Inserting this in the equation for $\mathrm{NEP}$ we get:

$$\begin{equation}
\mathrm{NEP}_{\tau=0.5\mathrm{s},\mathrm{ph}}=\sqrt{2\eta_0\mathrm{PSD}h\nu_0\left(1+\eta_0\mathrm{PSD}\right)\Delta\nu}
\end{equation}$$

If we assume the $\mathrm{PSD}$ to be flat, for a box filter the power spectrum multiplied by the bandwidth multiplied by the efficiency is obviously the power on the detector.

$$\begin{equation}
P_\mathrm{KID}=\eta_0\mathrm{PSD}\Delta\nu_0
\end{equation}$$

rearranging we get:

$$\begin{equation}
\mathrm{NEP}_{\tau=0.5\mathrm{s},\mathrm{ph}}=\sqrt{2P_\mathrm{KID}h\nu_0 + 2\frac{P^2_\mathrm{KID}}{\Delta\nu}}
\end{equation}$$

Which is in agreement with {% cite endo_2019%}.

In [18]:
#hide
nu = np.linspace(348.6,351.4,41)
#box_filter=(nu>125)*(nu<=175)/50
box_filter=np.zeros(41)
box_filter[20]=np.pi/4
lorentzian=cauchy.pdf(nu,350,0.35)*0.35*np.pi
df=pd.DataFrame({'nu':nu,'box filter':box_filter,'Lorentzian filter':lorentzian})

df.set_index('nu',inplace=True)

fig = px.bar(df,
             labels={'nu':'$\\nu\:\mathrm{[GHz]}$','value': '$\eta$','variable': 'Filter model'},
             barmode='overlay',
             template='plotly_white')
fig.update_layout(xaxis={'range':[348.9,351.1]})
fig.update_traces(width=0.07)
fig.data[0].width=0.7

for trace in fig.data:
    trace.hovertemplate= '<b>' + trace.name + '</b><br>bin frequency: %{x}<br>Efficiency: %{y:.2f}<extra></extra>'

fig.update_layout(hoverlabel = {'bgcolor': "white", 'font_size': 14})

#fig.show()

## Generation-Recombination Noise

Besides photon noise (both Poisson and bunching), another type of noise adds to our $\mathrm{NEP}$: recombination noise. This is a type of noise that occurs in superconductor-based pair-breaking photon detectors. As mentioned before, incoming photons generate quasiparticles in the detector, which later recombine in a cooper pair {% cite visser_2014 %}. For our purposes we will take the noise equivalent power of recombination noise $\mathrm{NEP_R}$ as given by {% cite endo_2019%}:

$$\begin{equation}
\mathrm{NEP_{R}}=\sqrt{4\Delta_\mathrm{Al}\frac{P_\mathrm{KID}}{\eta_\mathrm{pb}}}
\end{equation}$$

Even though the amount of quasiparticles generated is proportional to the incoming power, the recombination noise happens randomly and is therefore an uncorrelated process. This means we can add the $\mathrm{NEP_R}$ and $\mathrm{NEP_{ph}}$ together by quadrature addition, creating a total noise equivalent power given by:

$$\begin{equation}
\mathrm{NEP}_{\tau=0.5\mathrm{s}} = \sqrt{\mathrm{NEP}_{\tau=0.5\mathrm{s},\mathrm{ph}}^2 + \mathrm{NEP_{R}}^2}
\end{equation}$$

$$\begin{equation}
\mathrm{NEP}_{\tau=0.5\mathrm{s}}=\sqrt{2P_\mathrm{KID}h\nu_0 + 2\frac{P_\mathrm{KID}^2}{\Delta\nu}+4\Delta_\mathrm{Al}\frac{P_\mathrm{KID}}{\eta_\mathrm{pb}}}
\end{equation}$$

Finally, quasiparticles also generate randomly without any incoming photons, eg when the detector is kept in total darkness{%cite visser_2014_2%}. This is called *Generation-Recombination noise* and is negligible compared to the photon noise and photon-induced recombination noise as can be seen in the figure below:

![Generation-Recombination noise compared to photon noise](images/recombination_noise.png){%cite visser_2014_2%}*The noise induced by spontaneous Generation and recombination of quasiparticles is negligible compared to photon noise.*

Because this difference is negligible, I therefore won't model it.

<div style="display: flex;justify-content: space-between">
    <a href="/thesis/deshima">
        <div style="display: inline;font-size: x-large;padding: .5em;width:auto;" class="Box box-shadow-medium rounded-1 col-12">Previous Chapter</div>
    </a>
    <a href="/thesis/the-model">
        <div style="display: inline;font-size: x-large;padding: .5em;width:auto;" class="Box box-shadow-medium rounded-1 col-12">Next Chapter</div>
    </a>
</div>

## Bibliography
{% bibliography --cited %}